<a href="https://colab.research.google.com/github/mahlernim/ama484_2023_python_course/blob/main/2023%EB%94%94%EB%94%A4%EB%8F%8C09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# access_token 받아오기
# grant_type는 인증코드를 이용
# 60일에 한 번만 실행하면 되는 코드
# 오류 발생시 PPT 20번 내용을 다시 수행해서 인증코드를 재발급 받기
import requests, re, json
url = 'https://kauth.kakao.com/oauth/token'
d = {
  'grant_type': 'authorization_code',
  'client_id': 'd9e7430ae851a7b60ec9aff5ba8ea42c',
  'redirect_uri': 'https://example.com/oauth',
  'code': 'llJkoBVMRC2YUdPtX_8VMkhowgpyCwlaIHaZrIO2r4UsG3bqu5rlOaT8u-F4VUUjfDl5Lgo9dNsAAAGH3wWOxA'
}
response = requests.post(url, data=d)
tokens = response.json()
print(tokens)

# access_token, refresh_token 파일로 저장해두기
with open('access_token.txt', 'w') as f:
  f.write(tokens['access_token'])
with open('refresh_token.txt', 'w') as f:
  f.write(tokens['refresh_token'])



{'access_token': 'cdrNmexD3k36USMjpccxcDoEivnyYLRlOv9enqevCiolEAAAAYffDeXS', 'token_type': 'bearer', 'refresh_token': 'WdJIPddPAsHwTp_5pbMotfLVuVezdX7OyVeZNn_LCiolEAAAAYffDeXQ', 'expires_in': 21599, 'scope': 'talk_message profile_nickname', 'refresh_token_expires_in': 5183999}


In [ ]:
# 저장된 access_token 가져오기
with open('access_token.txt', 'r') as f:
  access_token = f.read()

# 메시지 보내기 API
url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'
h = {'Authorization': f'Bearer {access_token}'}
text = "토큰을 이용해보자 http://ubt.inje.ac.kr"
web_url = "https://example.com" # url 필수라서 의미 없는 url 추가
t = json.dumps({"object_type": "text", "text": text, "link": {"web_url": web_url}})
d = {"template_object": t}
response = requests.post(url, headers=h, data=d)
print(response.text)



{"result_code":0}


In [ ]:
# 60일간 유효한 refresh_token으로 access_token을 발급받는 과정
# 저장된 refresh_token 가져오기
with open('refresh_token.txt', 'r') as f:
  refresh_token = f.read()

# refresh_token 실행해서 access_token 갱신하기
import requests
url = 'https://kauth.kakao.com/oauth/token'
d = {
  'grant_type': 'refresh_token',
  'client_id': 'd9e7430ae851a7b60ec9aff5ba8ea42c',
  'redirect_uri': 'https://example.com/oauth',
  'refresh_token': refresh_token
}
response = requests.post(url, data=d)
tokens = response.json()
print(tokens)

# 새로운 access_token 덮어쓰기
with open('access_token.txt', 'w') as f:
  f.write(tokens['access_token'])



{'access_token': 'fqZussDm8VtpMAzSeDn6a1G0FYHC_d2SwyPndcBRCinI2gAAAYffLY3P', 'token_type': 'bearer', 'expires_in': 21599}


In [ ]:
client_id = "d9e7430ae851a7b60ec9aff5ba8ea42c"

In [ ]:
# refresh_token으로 access_token을 매번 새로 발급받아 사용하는 함수
import requests, json
def katok(text):
  global client_id
  # 저장된 refresh_token 가져오기
  with open('refresh_token.txt', 'r') as f:
    refresh_token = f.read()
  # refresh_token 실행해서 access_token 갱신하기
  import requests
  url = 'https://kauth.kakao.com/oauth/token'
  d = {
    'grant_type': 'refresh_token',
    'client_id': client_id,
    'redirect_uri': 'https://example.com/oauth',
    'refresh_token': refresh_token
  }
  response = requests.post(url, data=d)
  tokens = response.json()
  access_token = tokens['access_token']
  # 새로운 access_token 덮어쓰기
  with open('access_token.txt', 'w') as f:
    f.write(access_token)
  # 메시지 보내기 API
  url = 'https://kapi.kakao.com/v2/api/talk/memo/default/send'
  h = {'Authorization': f'Bearer {access_token}'}
  web_url = "https://example.com" # url 필수라서 의미 없는 url 추가
  t = json.dumps({"object_type": "text", "text": text, "link": {"web_url": web_url}})
  d = {"template_object": t}
  response = requests.post(url, headers=h, data=d)
  #print(response.text)
katok("hi!")



{"result_code":0}


In [ ]:
import requests
from bs4 import BeautifulSoup
url = "https://www.inje.ac.kr/kor/Template/Bsub_page.asp?Ldir=board/S_Food&Lpage=s_food_view"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, "html.parser")
menu_tags = soup.find_all("td", attrs={"class": "tc"})
yoil = "월화수목금"
text = "[다인A]\n"
for i in range(0,5):
  menu = menu_tags[i+2].getText(" ")
  menu = re.sub("<[^>]+>", "", menu)
  menu = re.sub("\([^)]+\)", "", menu)
  menu = re.sub("\s{2,}", " ", menu)
  menu = menu.strip()
  text += f"{yoil[i]}: {menu}\n"
print(text)
katok(text)



/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.inje.ac.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[다인A]
월: 근로자의 날 휴무
화: 참치야채비빔밥 뚝배기미니우동 해쉬포테이토+케찹
수: 묵은지돈육찜 감자채볶음 구이김+유장
목: 마늘보쌈정식 상추,깻잎+쌈장 쌈무,파절이 시락국/흑미밥
금: 어린이날 휴무

{"result_code":0}
